In [15]:
__author__ = 'matiasalvo'

import pandas as pd
from geopy import distance
import seaborn as sns
from sklearn.model_selection import train_test_split

products= pd.read_csv("data/order_products.csv")
orders = pd.read_csv("data/orders.csv")
shoppers = pd.read_csv("data/shoppers.csv")
store = pd.read_csv("data/storebranch.csv")

print(products.columns)
print(orders.columns)
print(shoppers.columns)
print(store.columns)

Index(['order_id', 'product_id', 'quantity', 'buy_unit'], dtype='object')
Index(['order_id', 'lat', 'lng', 'promised_time', 'on_demand', 'shopper_id',
       'store_branch_id', 'total_minutes'],
      dtype='object')
Index(['shopper_id', 'seniority', 'found_rate', 'picking_speed',
       'accepted_rate', 'rating'],
      dtype='object')
Index(['store_branch_id', 'store_id', 'lat', 'lng'], dtype='object')


In [2]:
#There are null_values for some shoppers.  We will replace it with the average value of the column
print((shoppers.isnull().sum(axis = 0)))
shoppers = shoppers.fillna(shoppers.mean())

shopper_id         0
seniority          0
found_rate       101
picking_speed      0
accepted_rate     27
rating            84
dtype: int64


In [3]:
#We add dummy variables for each type of seniority
unique_seniority = shoppers.seniority.unique()
for (i,key) in enumerate(unique_seniority):
    shoppers["seniority {}".format(i)] = shoppers.apply(lambda row: 1 if row.seniority == key else 0, axis=1)

In [4]:
#We create the columns of units and kgs
products["kg"] = products.apply(lambda row: row.quantity if row.buy_unit == "KG" else 0, axis = 1)
products["un"] = products.apply(lambda row: row.quantity if row.buy_unit == "un" else 0, axis = 1)

#For each order, we sum the units and kgs and we count the distinct products
products_gb = products.groupby("order_id").agg({"product_id":"count", 
                                                "quantity":"sum","kg":"sum"}).reset_index().rename(columns={"product_id":"distinct_products"})

In [5]:
#We join the datasets
df = orders.set_index("shopper_id").join(shoppers.set_index("shopper_id")).reset_index()
df = df.set_index("store_branch_id").join(store.set_index("store_branch_id"),lsuffix="_client",rsuffix="_store").reset_index()
df = df.set_index("order_id").join(products_gb.set_index("order_id")).reset_index()

In [6]:
#There are 22 orders without products.  As it is only a few, we are just going to drop them
df.isnull().sum(axis = 0)
df.dropna(subset=['kg'], how='all', inplace=True)
df.isnull().sum(axis = 0)

order_id                0
store_branch_id         0
shopper_id              0
lat_client              0
lng_client              0
promised_time           0
on_demand               0
total_minutes        1995
seniority               0
found_rate              0
picking_speed           0
accepted_rate           0
rating                  0
seniority 0             0
seniority 1             0
seniority 2             0
seniority 3             0
store_id                0
lat_store               0
lng_store               0
distinct_products       0
quantity                0
kg                      0
dtype: int64

In [7]:
#Calculate the distance between shop and customer
df["distance"] = df.apply(lambda row: distance.geodesic((row.lat_client,row.lng_client),(row.lat_store,row.lng_store)).km, axis=1)

In [8]:
#We create a few variables that may be useful
df["quantity_ratio"] = df["quantity"]/df["picking_speed"]
df["distinct_ratio"] = df["distinct_products"]/df["picking_speed"]
df["kg_ratio"] = df["kg"]/df["picking_speed"]
df["on_demand"] = df.on_demand.map(lambda x: int(x))

In [9]:
#We add dummy variable for store_branch_id and store_id
dummies = pd.get_dummies(df["store_branch_id"],prefix="store_branch")
# dummies2 = pd.get_dummies(df["store_id"],prefix="store_") #I tried these dummies and is worst for the models
df = pd.concat([df,dummies],axis=1)

In [10]:
#We normalize the variables so gradient descent works faster and regularization terms penalize
#each variable with the same weights
def normalize(df,columns):
    normalized = {x:{"mean":0, "std":0} for x in to_norm}
    for key in columns:
        mean = df[key].mean()
        std = df[key].std()
        normalized[key]["mean"] = mean
        normalized[key]["std"] = std
        df[key] = (df[key]-mean)/std
    return df, normalized

In [11]:
def denormalize(df,normalized):
    for key,value in normalized.items():
        if key != "total_minutes":
            df[key] = df[key]*value["std"] + value["mean"]
    return df

In [12]:
#Define columns to normalize, so the linear regression works faster
to_norm = ["found_rate", "picking_speed", "accepted_rate", "rating", "distinct_products",
           "quantity", "kg", "distance","quantity_ratio", "distinct_ratio", "kg_ratio"]
# We normalize the variables
df, normalized = normalize(df, to_norm)

In [13]:
#Reset the index and shuffle in case it was ordered (to get same distribution of train and test)
df = df.sample(frac=1, random_state=30).reset_index(drop=True)
#Dump the dataset 
df.to_pickle("data/regression_data.pkl")

In [31]:
#Split null and not null total_minutes entries, define training and test sets and drop columns we wont use
not_nulls = df[df.total_minutes.notnull()].drop(columns=["order_id","lat_client","lng_client","promised_time",
                            "seniority", "store_id","lat_store","lng_store","shopper_id", "store_branch_id"]
                                                ,axis=1).reset_index(drop=True)
y = not_nulls.total_minutes
x = not_nulls.drop(columns=["total_minutes"])
train_x, val_test_x, train_y, val_test_y = train_test_split(x,y,test_size = 0.3, shuffle=False)
val_x, test_x, val_y, test_y = train_test_split(val_test_x,val_test_y,test_size = 0.5, shuffle=False)


In [36]:
nulls = df[df.total_minutes.isna()].drop(columns=["lat_client","lng_client","promised_time",
                            "seniority", "store_id","lat_store","lng_store","shopper_id","store_branch_id"],axis=1).reset_index(drop=True)
y_nulls = nulls[["order_id"]]
x_nulls = nulls.drop(columns=["total_minutes", "order_id"])

In [38]:
train_x.to_pickle("data/train_x.pkl")
val_x.to_pickle("data/val_x.pkl")
test_x.to_pickle("data/test_x.pkl")
x_nulls.to_pickle("data/x_nulls.pkl")

train_y.to_pickle("data/train_y.pkl")
val_y.to_pickle("data/val_y.pkl")
test_y.to_pickle("data/test_y.pkl")
y_nulls.to_pickle("data/y_nulls.pkl")